In [220]:
import pandas as pd

In [221]:
ins = pd.read_csv('InstallmentsTable.csv')
ins.set_index(['borrower_id', 'loan_id'], inplace=True)

## Check Which Loans Have Defaulted  (Miss 2 installments In a row)

In [ ]:
#Mask for maximum lag
lag_mask = ins.groupby(level=1).shift(1).due_date.isnull()
#Mask for defaults (must occur beyond lag window if at beginning of installment series)
default_mask = (#ins.groupby(level=1).shift(2).paid_amount.isnull() & 
                ins.groupby(level=1).shift(1).paid_amount.isnull() & 
                ins.paid_amount.isnull() & ~lag_mask)

## Calculate Default Date

In [ ]:
defaults = ins[default_mask].groupby(level = ['borrower_id', 'loan_id']).agg({'due_date':'min'})
defaults.columns = ['default_date']

In [ ]:
targets = (ins.
join(defaults[['default_date']]).
groupby(level = ['borrower_id', 'loan_id']).
agg({'default_date':{'default_date': 'min', 'default_flag':lambda x: max(~x.isnull())}}))

targets.columns = targets.columns.droplevel()

targets.head(10)

In [ ]:
targets.to_csv('defaults.csv')

In [219]:
#check calculation
#ins[ins.index.get_level_values('borrower_id') == 131][['due_date', 'paid_amount']]